## Adrien Design Notebook

This Jupyter notebook contains an interactive environment where Adrien design features can be explored. It will be synced with the most recent Adrien library build.

In [3]:
#load "Adrien.Import.fsx"

open Adrien.Expression
open Adrien.Tree

let f (x:Expression) = sin x + constant 2.0f
(tree f).ToString()

"Tree {Parent = 0;
      Id = 1;
      Op = Add;
      Left = ValueNode {Id = 2;
                        Shape = Symbol;
                        Format = Nan;
                        Parent = 1;
                        Data = "x1";};
      Right = Some (ValueNode {Id = 3;
                               Shape = Scalar;
                               Format = Float32;
                               Parent = 1;
                               Data = 2.0f;});}"